## Getting Started with Mayavoz

#### Contents:
- [How to do inference using pretrained model](#inference)
- [How to train your custom model](#basictrain)

### Install Mayavoz

In [ ]:
! pip install -q mayavoz 

<div id="inference"></div>

###  Pretrained Model

To start using pretrained model,select any of the available recipes from [here](). 
For this exercice I am selecting [mayavoz/waveunet]()

- Mayavoz supports multiple input and output format. Input for inference can be in any of the below format
    - audio file path
    - numpy audio data
    - torch tensor audio data
    
It auto-detects the input format and does inference for you.
    
At the moment mayavoz only accepts single audio input

**Load model**

In [ ]:

from mayavoz import Mayamodel
model = Mayamodel.from_pretrained("mayavoz/waveunet")


**Inference using file path**

In [ ]:
file = "myvoice.wav"
audio = model.enhance("myvoice.wav")
audio.shape

**Inference using torch tensor**


In [ ]:
audio_tensor = torch.rand(1,1,32000) ## random audio data
audio = model.enhance(audio_tensor)
audio.shape

- if you want to save the output, just pass `save_output=True`

In [ ]:
audio = model.enhance("myvoice.wav",save_output=True)

In [ ]:
from Ipython.audio import Audio

Audio("myvoice_cleaned.wav",rate=SAMPLING_RATE)

<div id="basictrain"></div>


## Training your own custom Model

There are two ways of doing this

* [Using mayavoz framework ](#code)
* [Using mayavoz command line tool ](#cli)




<div id="code"></div>

**Using Mayavoz framwork** [Basic]
- Prepapare dataloader
- import preferred model
- Train

Files is dataclass that helps your to organise your train/test file paths

In [ ]:
from mayavoz.utils import Files

name = "dataset_name"
root_dir = "root_directory_of_your_dataset"
files = Files(train_clean="train_cleanfiles_foldername",
         train_noisy="noisy_train_foldername",
        test_clean="clean_test_foldername",
         test_noisy="noisy_test_foldername")
duration = 4.0 
stride = None
sampling_rate = 16000

Now there are two types of `matching_function`
- `one_to_one` : In this one clean file will only have one corresponding noisy file. For example VCTK datasets
- `one_to_many` : In this one clean file will only have one corresponding noisy file. For example DNS dataset.

In [ ]:
mapping_function = "one_to_one"


In [ ]:
from mayavoz.dataset import MayaDataset
dataset = MayaDataset(
            name=name,
            root_dir=root_dir,
            files=files,
            duration=duration,
            stride=stride,
            sampling_rate=sampling_rate
        )


In [ ]:
from mayavoz.models import Demucs
model = Demucs(dataset=dataset, loss="mae")


In [2]:
import pytorch_lightning as pl

In [ ]:
trainer = pl.Trainer(model)
trainer.fit(max_epochs=1)

**mayavoz model and dataset are highly customazibale**, see [here]() for advanced usage

<div id="cli"></div>


## Mayavoz CLI

In [ ]:
! pip install mayavoz[cli]

### TL;DR
Calling the following command would train mayavoz Demucs model on DNS-2020 dataset.

```bash
mayavoz-train \
    model=Demucs \
    Demucs.sampling_rate=16000 \
    dataset=DNS-2020 \
    DNS-2020.name = "dns-2020" \
    DNS-2020.root_dir="your_root_dir" \
    DNS-2020.train_clean="" \
    DNS-2020.train_noisy="" \
    DNS-2020.test_clean="" \
    DNS-2020.test_noisy="" \
    DNS-2020.sampling_rate=16000 \
    DNS-2020.duration=2.0 \
    traine=default \ 
    default.max_epochs=1 \

```

This is more or less equaivalent to below code

In [ ]:
from mayavoz.utils import Files
from mayavoz.data import MayaDataset
from mayavoz.models import Demucs

files = Files(
        train_clean="",
        train_noisy="",
        test_clean="",
        test_noisy=""
)
dataset = MayaDataset(
                    name='dns-2020'
                    root_dir="your_root_dir",
                    files=files,
                    sampling_rate=16000,
                    duration=2.0)
model = Demucs(dataset=dataset,sampling_rate=16000)
trainer = Trainer(max_epochs=1)
trainer.fit(model)

Hydra-based configuration
mayavoz-train relies on Hydra to configure the training process. Adding --cfg job option to the previous command will let you know about the actual configuration used for training:

```bash
mayavoz-train --cfg job \
    model=Demucs \
    Demucs.sampling_rate=16000 \
    dataset=DNS-2020

```

```yaml
_target_: enhancer.models.demucs.Demucs
num_channels: 1
resample: 4
sampling_rate : 16000

encoder_decoder:
  depth: 4
  initial_output_channels: 64
  
[...]
```

To change the sampling_rate, you can 

```bash
mayavoz-train \
    model=Demucs model.sampling_rate=16000 \
    dataset=DNS-2020

```